# Scraping and Cleaning High-Level College Data

In this project, we'll be parsing college data from [CollegeData.com](https://www.collegedata.com/en/).  The site provides a wealth of information for prospective U.S.-based college students in an effort to help them with their decision-making process (providing data relating to areas such as financials, student population and statistics, satisfaction rates, etc).  It's a reasonably full-featured platform, overall, that provides users the ability to dive into the data, track deadlines, and find strong school matches for themselves; among other things.

We'll be pulling and parsing HTML data from CollegeData's search feature, focusing on compiling the high-level data provided initially as search results by the site (versus diving into individual college pages for more granular data - certainly a candidate for a subsequent project).  Once we parse out the information we're interested in, we will organize and clean it using pandas.

## Pulling the HTML Code & Parsing The Data

For the sake of expediency, avoiding possible future issues with changes to the HTML structure of CollegeData's search results page, and accounting for the fact that the search results gate some of the data based on the lack of a request verification token (normally attained through being logged in to a free account on the website); we've already saved a copy of the full HTML file (`collegedata.html`) that was pulled using Python's `requests` package.

For posterity: the HTML was retrieved by sending a `GET` request to `https://www.collegedata.com/en/explore-colleges/college-search/SearchByPreference/?SearchByPreference.SearchType=1&SearchByPreference.CollegeName=*` with a request verification token included within the request headers.  Note that this URL provides nothing other than `*` as search criteria, allowing us to pull results that include all available schools in CollegeData's database.

### Opening the HTML & Initializing the Parser

We'll begin by opening our HTML file and using the BeautifulSoup package to create an HTML parser object, which we will use to extract the data we're interested in.  We'll also import the `re` package to use in combination with BeautifulSoup in selecting target elements.

In [3]:
from bs4 import BeautifulSoup
import re

with open("collegedata.html") as html_file:
    html_data = BeautifulSoup(html_file, "html.parser")

On CollegeData's search results page, results are organized into a table with 3 separate tabs of data (Vitals, Financial, and Students) and a paginated list of schools making up the table rows.  The "College Chances" and locker icon columns are relevant to the site's services, but not to us.  The results page indicates that, by running a search that captures all schools in CollegeData's database, we get data on 1966 schools total.

![title](collegedataresults.png)

Upon brief inspection of the HTML code, we notice that each table row element that represents a school's data contains the `data-school-id` attribute along with an ID value.  Let's parse the entire tree for every element containing this attribute.

In [5]:
data_by_ids = html_data.find_all(attrs={"data-school-id": True})
len(data_by_ids)

5898

There are 5898 items contained in this collection, which suggests that each school is represented in the code 3 times.  This makes sense, given that there are 3 tabs of data in the table, but let's confirm by looking at the number of _unique_ ID values (we should expect 1966).

In [13]:
ids = set([int(item["data-school-id"]) for item in data_by_ids])
len(ids)

1966

Upon further inspection of the HTML code, we notice that each tab appears to be represented as a distinct table entirely, each containing an element ID that uses the `colleges_table_` prefex.  Let's check how many of these there are.

In [11]:
data_tables = data.find_all(id=re.compile("colleges_table_"))

for table in data_tables:
    print(table['id'])

colleges_table_vitals
colleges_table_financial
colleges_table_students


Confirmed: there are 3 separate tables, each with an ID beginning with `colleges_table_` and ending with the tab label.  Given this structure, we'll start out by extracting the data we want into 3 separate collections, and then combine them.

### Extracting the Table Rows

Let's create 3 lists to represent the row elements from each table, and populate them with the table row elements (again, each representing a school) by using the same `data-school-id` attribute we used above.

In [15]:
table_rows = {
    "vitals": [],
    "financial": [],
    "students": []
}

# iterate through each table element, searching it for all elements with data-school-id attributes and saving to the appropriate list
for table in data_tables:
    table_name = table["id"].replace("colleges_table_", "")
    row_elements = table.find_all(attrs={"data-school-id": True})
    table_rows[table_name] = row_elements
    
len(table_rows["vitals"]) # check the length of one of our collections, to ensure it's 1966, as expected

1966

### Extracting Values From Table Cells

Now that we have our 3 lists of table rows, let's iterate through them and extract the individual values for each of the columns of relevant data.  We'll create a helper function to extract a cell value, which we can use for every column other than the college name and CollegeData assigned school ID value (the latter of which we'll capture now, but won't need later).

In [16]:
# helper function: extracts text from element passed to it
def extract_value(cell):
    return cell.find_all(string=True)[2].strip()

# extract values for the rows in all 3 tables
table_row_values = {}
for table in table_rows.keys():
    table_items = []
    for row in table_rows[table]:
        row_data = []
        row_cells = row.find_all("td")
        row_data.append(row_cells[0].find(href=re.compile("/college")).string)
        row_data.append(row_cells[1].find(attrs={"data-schoolidvalue":True})["data-schoolidvalue"]) # we'll need these assigned school_id values for merging data later
        for i in range(2, len(row_cells)):
            row_data.append(extract_value(row_cells[i]))
        table_items.append(row_data)
    table_row_values[table] = table_items

Let's check that the values for each table have been correctly extracted by examining the first row's from each.

In [17]:
print(table_row_values["vitals"][0])
print(table_row_values["financial"][0])
print(table_row_values["students"][0])

['West Texas A&M University', '949', 'Canyon', 'TX', '7,394', 'Pub', 'Mod', '64%', '25.2%']
['West Texas A&M University', '949', '', '$21,426', '$23,012', '62%', '55%', '$23,670']
['West Texas A&M University', '949', 'Coed', '0.5%', '4.9%', '1.9%', '29%', '1.8%']


We've correctly extracted all of the values we're interested in.

## Converting to DataFrames

Now let's move on to compiling our data into pandas dataframes.  We'll start out, keeping it simple, by using the same column names as the CollegeData search results tabs (but convert them to snake-case, to conform to Python standards).

In [21]:
import pandas as pd

table_columns = {
    "vitals": ["name", "school_id", "city", "state", "size", "type", "entrance_difficulty", "freshman_satisfaction", "grad_rate"],
    "financial": ["name", "school_id", "your_net_price", "resident_coa", "nonresident_coa", "need_met", "merit_aid", "student_debt"],
    "students": ["name", "school_id", "gender_mix", "american_indian", "african_american", "asian_pacific_islander", "hispanic", "intl"]
}

vitals_data = pd.DataFrame(table_row_values["vitals"], columns = table_columns["vitals"])
financial_data = pd.DataFrame(table_row_values["financial"], columns = table_columns["financial"])
students_data = pd.DataFrame(table_row_values["students"], columns = table_columns["students"])

In [22]:
vitals_data.head()

,name,school_id,city,state,size,type,entrance_difficulty,freshman_satisfaction,grad_rate
0,West Texas A&M University,949,Canyon,TX,"7,394",Pub,Mod,64%,25.2%
1,Rose-Hulman Institute of Technology,883,Terre Haute,IN,"2,168",Priv,Very,91.2%,66.5%
2,Dominican College,886,Orangeburg,NY,"1,425",Priv,Non,74%,28.7%
3,University of Puget Sound,859,Tacoma,WA,"2,364",Priv,Mod,80.7%,65.8%
4,Champlain College,1270,Burlington,VT,"2,129",Priv,Mod,78%,53.9%


In [23]:
financial_data.head()

,name,school_id,your_net_price,resident_coa,nonresident_coa,need_met,merit_aid,student_debt
0,West Texas A&M University,949,,"$21,426","$23,012",62%,55%,"$23,670"
1,Rose-Hulman Institute of Technology,883,,"$70,401","$70,401",Not reported,Not Reported,"$45,345"
2,Dominican College,886,,"$46,600","$46,600",71%,17%,"$32,527"
3,University of Puget Sound,859,,"$68,146","$68,146",78%,44%,"$32,999"
4,Champlain College,1270,,"$61,012","$61,012",71%,22%,"$35,383"


In [24]:
students_data.head()

,name,school_id,gender_mix,american_indian,african_american,asian_pacific_islander,hispanic,intl
0,West Texas A&M University,949,Coed,0.5%,4.9%,1.9%,29%,1.8%
1,Rose-Hulman Institute of Technology,883,,0.2%,3.2%,6.1%,5.3%,14.6%
2,Dominican College,886,Coed,0%,15.2%,6.8%,30.8%,1.4%
3,University of Puget Sound,859,Coed,0.1%,1.8%,7%,8.8%,0.4%
4,Champlain College,1270,Coed,0.2%,2.7%,3%,6.7%,0.8%


### Merging the Data

Now that we've created our 3 separate dataframes, we can merge them into one.  Because all schools are represented in each of the 3 dataframes, we can confidently use an inner join.  We'll also merge on both the `school_id` and `name` values, since not only do both exist in all 3 dataframes, but it allows us to avoid any column duplication.

In [25]:
merged_data = vitals_data.merge(financial_data, how="inner", on=["name", "school_id"])
merged_data = merged_data.merge(students_data, how="inner", on=["name", "school_id"])

merged_data.head()

,name,school_id,city,state,size,type,entrance_difficulty,freshman_satisfaction,grad_rate,your_net_price,...,nonresident_coa,need_met,merit_aid,student_debt,gender_mix,american_indian,african_american,asian_pacific_islander,hispanic,intl
0,West Texas A&M University,949,Canyon,TX,"7,394",Pub,Mod,64%,25.2%,,...,"$23,012",62%,55%,"$23,670",Coed,0.5%,4.9%,1.9%,29%,1.8%
1,Rose-Hulman Institute of Technology,883,Terre Haute,IN,"2,168",Priv,Very,91.2%,66.5%,,...,"$70,401",Not reported,Not Reported,"$45,345",,0.2%,3.2%,6.1%,5.3%,14.6%
2,Dominican College,886,Orangeburg,NY,"1,425",Priv,Non,74%,28.7%,,...,"$46,600",71%,17%,"$32,527",Coed,0%,15.2%,6.8%,30.8%,1.4%
3,University of Puget Sound,859,Tacoma,WA,"2,364",Priv,Mod,80.7%,65.8%,,...,"$68,146",78%,44%,"$32,999",Coed,0.1%,1.8%,7%,8.8%,0.4%
4,Champlain College,1270,Burlington,VT,"2,129",Priv,Mod,78%,53.9%,,...,"$61,012",71%,22%,"$35,383",Coed,0.2%,2.7%,3%,6.7%,0.8%


In [26]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1966 entries, 0 to 1965
Data columns (total 21 columns):
name                      1966 non-null object
school_id                 1966 non-null object
city                      1966 non-null object
state                     1966 non-null object
size                      1966 non-null object
type                      1966 non-null object
entrance_difficulty       1966 non-null object
freshman_satisfaction     1966 non-null object
grad_rate                 1966 non-null object
your_net_price            1966 non-null object
resident_coa              1966 non-null object
nonresident_coa           1966 non-null object
need_met                  1966 non-null object
merit_aid                 1966 non-null object
student_debt              1966 non-null object
gender_mix                1966 non-null object
american_indian           1966 non-null object
african_american          1966 non-null object
asian_pacific_islander    1966 non-null object


We've successfully merged our data into one dataframe.

It appears that there are no missing values anywhere, and that every column contains string values - we'll come back to this a little later.

## Cleaning the Data

### Removing Irrelevant Columns

There are two columns that now stand out as unnecessary at this point: `school_id` (ID values assigned by CollegeData) and `your_net_price` (a financial calculation column only used as part of CollegeData's services).  Let's drop them.

In [27]:
merged_data = merged_data.drop(["your_net_price", "school_id"], axis = 1)

### Adjusting Column Names

Let's modify a few column names for clarity.

In [40]:
merged_data.rename(columns={
    "size": "n_students",
    "entrance_difficulty": "acceptance_difficulty",
    "resident_coa": "resident_cost",
    "nonresident_coa": "nonresident_cost",
    "student_debt": "avg_grad_debt",
    "american_indian": "pct_american_indian",
    "african_american": "pct_african_american",
    "asian_pacific_islander": "pct_asian_pacific_islander",
    "hispanic": "pct_hispanic",
    "intl": "pct_international"
}, inplace=True)

merged_data.columns

Index(['name', 'city', 'state', 'n_students', 'type', 'acceptance_difficulty',
       'freshman_satisfaction', 'grad_rate', 'resident_cost',
       'nonresident_cost', 'need_met', 'merit_aid', 'avg_grad_debt',
       'gender_mix', 'pct_american_indian', 'pct_african_american',
       'pct_asian_pacific_islander', 'pct_hispanic', 'pct_international'],
      dtype='object')

### Handling Missing Values

As was observed earlier, on first glance, it appears that our dataset contains no missing values.  However, given that our data originated from parsing HTML, it is all in string form.  Thus, missing values could still exist, such as in the form of an empty string.  From browsing through the original search results on the CollegeData.com website, we know that some schools were missing data, and the missing values were represented in a few different ways.

To start, let's look into how many empty strings exist for each of our columns.

In [41]:
for col in merged_data.columns:
    print("Empty strings in " + col + ":", len(merged_data[merged_data[col] == ""]))

Empty strings in name: 0
Empty strings in city: 0
Empty strings in state: 0
Empty strings in n_students: 0
Empty strings in type: 7
Empty strings in acceptance_difficulty: 180
Empty strings in freshman_satisfaction: 0
Empty strings in grad_rate: 0
Empty strings in resident_cost: 0
Empty strings in nonresident_cost: 0
Empty strings in need_met: 0
Empty strings in merit_aid: 0
Empty strings in avg_grad_debt: 0
Empty strings in gender_mix: 77
Empty strings in pct_american_indian: 0
Empty strings in pct_african_american: 0
Empty strings in pct_asian_pacific_islander: 0
Empty strings in pct_hispanic: 0
Empty strings in pct_international: 0


We see that the columns `type` (7), `acceptance_difficulty` (180), and `gender` (77) each contain missing (empty string) values.  To fix that, we'll replace the empty strings with `Not Reported`, to stay consistent with other existing entries in the data.

In [42]:
merged_data.loc[merged_data["type"] == "", "type"] = "Not Reported"
merged_data["type"].value_counts()

Priv            1193
Pub              594
Proft            172
Not Reported       7
Name: type, dtype: int64

In [43]:
merged_data.loc[merged_data["acceptance_difficulty"] == "", "acceptance_difficulty"] = "Not Reported"
merged_data["acceptance_difficulty"].value_counts()

Mod             1078
Min              317
Not Reported     180
Non              175
Very             157
Most              59
Name: acceptance_difficulty, dtype: int64

In [46]:
merged_data.loc[merged_data["gender_mix"] == "", "gender_mix"] = "Not Reported"
merged_data["gender_mix"].value_counts()

Coed            1821
Not Reported      77
Men               56
Women             12
Name: gender_mix, dtype: int64